In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/home/kali/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

!unzip celeba-dataset.zip

In [1]:
import os
import numpy as np
import cv2
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as patches



# Set the path to the CSV files containing the annotations and images
attr_file = os.path.join("list_attr_celeba.csv")
bbox_file = os.path.join("list_bbox_celeba.csv")
landmarks_file = os.path.join("list_landmarks_align_celeba.csv")
images_dir = os.path.join("img_align_celeba/img_align_celeba/")



# Read the annotations from the CSV files
with open(attr_file, 'r') as f:
    reader = csv.reader(f)
    lines = list(reader)
    attr_names = lines[0]
    attr_data = lines[1:]

with open(bbox_file, 'r') as f:
    reader = csv.reader(f)
    lines = list(reader)
    bbox_data = lines[1:]

with open(landmarks_file, 'r') as f:
    reader = csv.reader(f)
    lines = list(reader)
    landmarks_data = lines[1:]


# Create a dictionary that maps the image name to the annotations
attr = {}
bbox = {}
landmarks = {}
for i in range(len(attr_data)):
    filename = attr_data[i][0]
    values = attr_data[i][1:]
    attr[filename] = values

for i in range(len(bbox_data)):
    filename = bbox_data[i][0]
    values = bbox_data[i][1:]
    bbox[filename] = values

for i in range(len(landmarks_data)):
    filename = landmarks_data[i][0]
    values = landmarks_data[i][1:]
    landmarks[filename] = values

    # Create a list of images that have a face, save them together with their annotations
face_images = []
for filename in attr:
    if attr[filename][20] == '1':
        face_images.append((filename, attr[filename], bbox[filename], landmarks[filename]))

# frontal face images
frontal_face_images = []
for filename in attr:
    if attr[filename][20] == '1' and attr[filename][21] == '-1':
        frontal_face_images.append((filename, attr[filename], bbox[filename], landmarks[filename]))

print(len(frontal_face_images)) 

# save the frontal face images in a folder and save the annotations in a csv file
with open('frontal_face_images.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'attr', 'bbox', 'landmarks'])
    for image in frontal_face_images:
        writer.writerow(image)


frontal_face_images = pd.read_csv('frontal_face_images.csv')
frontal_face_images.head()

"""
frontal_face_images columns:filename	attr	bbox	landmarks
"""

# Model to be used for face detection, Use CNN for better accuracy
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

# Data Preprocessing
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                      shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                    target_size = (64, 64),
                                                    batch_size = 32,
                                                    class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                                target_size = (64, 64),
                                                batch_size = 32,
                                                class_mode = 'binary')



    



ModuleNotFoundError: No module named 'cv2'